## https://flightaware.com/live/fleet/LAN


In [ ]:
from robobrowser import RoboBrowser
import plotly.plotly as py
import pandas as pd
import networkx as nx
from cachier import cachier
import geocoder
browser = RoboBrowser(history=True, parser='html5lib')

In [ ]:
url = 'https://flightaware.com/live/fleet/LAN'
browser.open(url)
table=browser.select('.prettyTable')[0]
table_code=table.__str__()
table_code[:1000]

In [ ]:
tdf=pd.read_html(table_code, header=1)[0]
tdf

## http://flightaware.com/live/fleet/AAL?;offset=20

In [15]:
browser = RoboBrowser(history=True, parser='html5lib')
base_url = 'https://flightaware.com/live/fleet/'
query = 'JAL' ## 
airlines = "LAN", "JAL", "DLH", "AAL", "CCA", 'AFR', 'IGO', 'MSR'
df = pd.DataFrame()

for query in airlines:
    print('-----------')
    print(query)
    for i in range(2):
        url = base_url + query + "?;offset=" + str(i*20)
        browser.open(url)
        table=browser.select('.prettyTable')[0]
        t=table.__str__()
        if 'Sorry' in t:
                break
        tdf=pd.read_html(t, header=1)[0]
        df = df.append(tdf)
        print(tdf.head(3))

-----------
LAN
     Ident  Type                                             Origin  \
0  LAN1137  A320                    Cerro Moreno Int'l (ANF / SCFA)   
1   LAN174  A320  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)   
2   LAN480  A321                         Jorge Newbery (AEP / SABE)   

                                         Destination        Departure  \
0  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)  Thu 09:43PM -03   
1                   Diego Aracena Int'l (IQQ / SCDA)  Thu 09:49PM -03   
2  Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)  Thu 08:39PM -03   

  EstimatedArrival Time  
0       Thu 11:11PM -03  
1       Fri 12:04AM -03  
2       Thu 10:43PM -03  
-----------
JAL
    Ident  Type                            Origin  \
0    JAL1  B77W         San Francisco Intl (KSFO)   
1   JAL11  B788     Dallas-Fort Worth Intl (KDFW)   
2  JAL110  B763  Osaka Int'l (Itami) (ITM / RJOO)   

                         Destination        Departure EstimatedArrival Ti

In [16]:
df.head()

,Ident,Type,Origin,Destination,Departure,EstimatedArrival Time
0,LAN1137,A320,Cerro Moreno Int'l (ANF / SCFA),Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Thu 09:43PM -03,Thu 11:11PM -03
1,LAN174,A320,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Diego Aracena Int'l (IQQ / SCDA),Thu 09:49PM -03,Fri 12:04AM -03
2,LAN480,A321,Jorge Newbery (AEP / SABE),Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Thu 08:39PM -03,Thu 10:43PM -03
3,LAN503,B789,Miami Intl (KMIA),Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Thu 09:09PM EDT,Fri 06:19AM -03
4,LAN530,B788,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Jorge Chavez Int'l (LIM / SPJC),Thu 08:25PM -03,Thu 10:00PM -05


In [20]:
from joblib import Memory
memory = Memory(cachedir='tmp', verbose=0)

@memory.cache
def get_latlng(query):
    g=geocoder.google(query)
    if g:
        return g.latlng
    else:
        return None, None

In [21]:

great_lines = []
airports = []
airport_data = []
seen=[]
great_lines_2=[]
airports = pd.DataFrame(columns=['label','lat','lon','size'])

    
i=0
for index,row in df.iterrows():
    origin = row.Origin
    destination = row.Destination
    olat, olon = get_latlng(origin)
    dlat, dlon = get_latlng(destination)
    if olat and dlat:
        if origin not in seen:
            seen.append(origin)
            print(origin, olat, olon)
        if destination not in seen:
            seen.append(destination)
            print(destination, dlat, dlon)
        great_lines.append(dict(
            type = 'scattergeo',
            lat = [ olat, dlat ],
            lon = [ olon, dlon ],
            hoverinfo = 'Ident',
                text = 'Ident',
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'rgba(255,0,0,0.2)',
                ),
                )
        )
        airports.loc[i]=[origin, olat, olon, 0.0]
        airports.loc[i+1]= [destination, dlat, dlon, 0.0]
        i+=2
     

Cerro Moreno Int'l (ANF / SCFA) -23.4486944 -70.4409348
Comodoro Arturo Merino Benitez Int'l (SCL / SCEL) -33.39013449999999 -70.794162
Diego Aracena Int'l (IQQ / SCDA) -20.5364084 -70.1805181
Jorge Newbery (AEP / SABE) -34.5580305 -58.4170088
Miami Intl (KMIA) 25.795865 -80.2870457
Jorge Chavez Int'l (LIM / SPJC) -12.0240527 -77.112036
John F Kennedy Intl (KJFK) 40.6413111 -73.77813909999999
Los Angeles Intl (KLAX) 33.9415889 -118.40853
Lic. Benito Juarez Int'l (MEX / MMMX) 19.4360762 -99.07190829999999
Barajas Int'l (MAD / LEMD) 40.4839361 -3.5679515
São Paulo-Guarulhos Int'l (GRU / SBGR) -23.4345529 -46.478126
Melbourne Tullamarine (MEL / YMML) -37.6690123 144.8410273
Carlos Ibanez Del Campo Int'l (PUQ / SCCI) -53.0050882 -70.8430601
Dallas-Fort Worth Intl (KDFW) 32.8998091 -97.0403352
Narita Int'l (NRT / RJAA) 35.7719867 140.3928501
Osaka Int'l (Itami) (ITM / RJOO) 34.7895799 135.4380895
Tokyo Int'l (Haneda) (HND / RJTT) 35.5493932 139.7798386
Aomori (AOJ / RJSA) 40.7382794 140.689

In [30]:
great_lines

[{'hoverinfo': 'Ident',
  'lat': [-23.4486944, -33.39013449999999],
  'line': {'color': 'rgba(255,0,0,0.2)', 'width': 1},
  'lon': [-70.4409348, -70.794162],
  'mode': 'lines',
  'text': 'Ident',
  'type': 'scattergeo'},
 {'hoverinfo': 'Ident',
  'lat': [-33.39013449999999, -20.5364084],
  'line': {'color': 'rgba(255,0,0,0.2)', 'width': 1},
  'lon': [-70.794162, -70.1805181],
  'mode': 'lines',
  'text': 'Ident',
  'type': 'scattergeo'},
 {'hoverinfo': 'Ident',
  'lat': [-34.5580305, -33.39013449999999],
  'line': {'color': 'rgba(255,0,0,0.2)', 'width': 1},
  'lon': [-58.4170088, -70.794162],
  'mode': 'lines',
  'text': 'Ident',
  'type': 'scattergeo'},
 {'hoverinfo': 'Ident',
  'lat': [25.795865, -33.39013449999999],
  'line': {'color': 'rgba(255,0,0,0.2)', 'width': 1},
  'lon': [-80.2870457, -70.794162],
  'mode': 'lines',
  'text': 'Ident',
  'type': 'scattergeo'},
 {'hoverinfo': 'Ident',
  'lat': [-33.39013449999999, -12.0240527],
  'line': {'color': 'rgba(255,0,0,0.2)', 'width': 

In [22]:
airports.head(20)

,label,lat,lon,size
0,Cerro Moreno Int'l (ANF / SCFA),-23.448694,-70.440935,0.0
1,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,0.0
2,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,0.0
3,Diego Aracena Int'l (IQQ / SCDA),-20.536408,-70.180518,0.0
4,Jorge Newbery (AEP / SABE),-34.558031,-58.417009,0.0
5,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,0.0
6,Miami Intl (KMIA),25.795865,-80.287046,0.0
7,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,0.0
8,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,0.0
9,Jorge Chavez Int'l (LIM / SPJC),-12.024053,-77.112036,0.0


In [23]:
df.head()

,Ident,Type,Origin,Destination,Departure,EstimatedArrival Time
0,LAN1137,A320,Cerro Moreno Int'l (ANF / SCFA),Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Thu 09:43PM -03,Thu 11:11PM -03
1,LAN174,A320,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Diego Aracena Int'l (IQQ / SCDA),Thu 09:49PM -03,Fri 12:04AM -03
2,LAN480,A321,Jorge Newbery (AEP / SABE),Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Thu 08:39PM -03,Thu 10:43PM -03
3,LAN503,B789,Miami Intl (KMIA),Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Thu 09:09PM EDT,Fri 06:19AM -03
4,LAN530,B788,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),Jorge Chavez Int'l (LIM / SPJC),Thu 08:25PM -03,Thu 10:00PM -05


In [24]:
import networkx as nx
G = nx.from_pandas_dataframe(df, 'Origin', 'Destination')

In [25]:
for node in G.nodes():
    print(node)

Cerro Moreno Int'l (ANF / SCFA)
Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)
Diego Aracena Int'l (IQQ / SCDA)
Jorge Newbery (AEP / SABE)
Miami Intl (KMIA)
Jorge Chavez Int'l (LIM / SPJC)
John F Kennedy Intl (KJFK)
Los Angeles Intl (KLAX)
Lic. Benito Juarez Int'l (MEX / MMMX)
Barajas Int'l (MAD / LEMD)
São Paulo-Guarulhos Int'l (GRU / SBGR)
Melbourne Tullamarine (MEL / YMML)
Carlos Ibanez Del Campo Int'l (PUQ / SCCI)
San Francisco Intl (KSFO)
Tokyo Int'l (Haneda) (HND / RJTT)
Dallas-Fort Worth Intl (KDFW)
Narita Int'l (NRT / RJAA)
Osaka Int'l (Itami) (ITM / RJOO)
Aomori (AOJ / RJSA)
Vancouver Int'l (CYVR)
Komatsu (KMQ / RJNK)
Beijing Capital Int'l (PEK / ZBAA)
Naha (OKA / ROAH)
Amami (ASJ / RJKA)
Kansai Int'l (KIX / RJBB)
New Chitose (CTS / RJCC)
Hiroshima (HIJ / RJOA)
Izumo (IZO / RJOC)
Fukuoka (FUK / RJFF)
Chubu Centrair Int'l (Centrair) (NGO / RJGG)
New Kitakyushu (KKJ / RJFR)
Singapore Changi (SIN / WSSS)
Frankfurt Int'l (FRA / EDDF)
London Heathrow (LHR / EGLL)
Matsuyama (MYJ 

In [26]:
for edge in G.edges():
    print(edge)

("Cerro Moreno Int'l (ANF / SCFA)", "Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)")
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", "Diego Aracena Int'l (IQQ / SCDA)")
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", 'Jorge Newbery (AEP / SABE)')
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", 'Miami Intl (KMIA)')
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", "Jorge Chavez Int'l (LIM / SPJC)")
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", 'John F Kennedy Intl (KJFK)')
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", "Lic. Benito Juarez Int'l (MEX / MMMX)")
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", "Barajas Int'l (MAD / LEMD)")
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", "São Paulo-Guarulhos Int'l (GRU / SBGR)")
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", 'Melbourne Tullamarine (MEL / YMML)')
("Comodoro Arturo Merino Benitez Int'l (SCL / SCEL)", "Carlos Ibanez Del Campo Int'l (PUQ / SCCI)")
("Comodoro Arturo Meri

## Betweenness Centrality
![Betweenness](betweenness.png)

In [27]:
centrality=nx.betweenness_centrality(G)
centrality

{'Abha Regional (AHB / OEAB)': 0.0,
 'Amami (ASJ / RJKA)': 0.0,
 'Aomori (AOJ / RJSA)': 0.0,
 'Austin-Bergstrom Intl (KAUS)': 0.0,
 'Baltimore/Washington Intl (KBWI)': 0.0,
 'Baotou (BAV / ZBOW)': 0.0,
 "Barajas Int'l (MAD / LEMD)": 0.0,
 "Barcelona Int'l (BCN / LEBL)": 0.0,
 "Beijing Capital Int'l (PEK / ZBAA)": 0.29648397165231527,
 "Ben Gurion Int'l (TLV / LLBG)": 0.0,
 "Bengaluru Int'l (BLR / VOBL)": 0.05161521252796426,
 'Biju Patnaik (BBI / VEBS)': 0.0,
 "Bole Int'l (ADD / HAAB)": 0.0,
 'Borg El Arab (HBE / HEBA)': 0.0,
 'Boston Logan Intl (KBOS)': 0.01706883234805849,
 "Cairo Int'l (CAI / HECA)": 0.13311355903114064,
 "Calicut Int'l (Karipur) (CCJ / VOCL)": 0.012975391498881432,
 "Cape Town Int'l (CPT / FACT)": 0.0009000121911689225,
 "Carlos Ibanez Del Campo Int'l (PUQ / SCCI)": 0.0,
 "Cerro Moreno Int'l (ANF / SCFA)": 0.0,
 "Changsha Huanghua Int'l (CSX / ZGHA)": 0.0,
 'Charles de Gaulle/Roissy (CDG / LFPG)': 0.437825815981941,
 'Charlotte/Douglas Intl (KCLT)': 0.0,
 "Chatrapa

In [28]:
for key, item in centrality.items():
    airports.loc[airports.label==key, 'size'] = (item*100)+10
airports.head(20)

,label,lat,lon,size
0,Cerro Moreno Int'l (ANF / SCFA),-23.448694,-70.440935,10.000000
1,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,20.416544
2,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,20.416544
3,Diego Aracena Int'l (IQQ / SCDA),-20.536408,-70.180518,10.000000
4,Jorge Newbery (AEP / SABE),-34.558031,-58.417009,10.000000
5,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,20.416544
6,Miami Intl (KMIA),25.795865,-80.287046,15.706312
7,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,20.416544
8,Comodoro Arturo Merino Benitez Int'l (SCL / SCEL),-33.390134,-70.794162,20.416544
9,Jorge Chavez Int'l (LIM / SPJC),-12.024053,-77.112036,10.219939


## https://plot.ly/

In [29]:
import plotly.plotly as py

airport_markers = [ dict(
        type = 'scattergeo',
        lon = airports['lon'],
        lat = airports['lat'],
        hoverinfo = 'label',
        text = airports['label'],
        mode = 'markers',
        marker = dict( 
            size=airports['size'], 
            color='red',   
        ))]
    
layout = dict(
        title = 'Current flight network',
        showlegend = False,  
        width = 800,
        height = 800,
        geo = dict(
            showland = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            landcolor = '#fff',
            oceancolor = '#eee',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            
        )
    )
    
fig = dict( data=great_lines+airport_markers, layout=layout )
py.iplot( fig, validate=False, filename='d3-globe' )